In [227]:
# %load_ext autoreload
%autoreload 2
import torch
from datasets import TripletAudio

K, MAX_CLOSE_NEG, MAX_FAR_NEG = 5, 15, 15
BATCH_SIZE = 1

triplet_train_dataset = TripletAudio(True, K, MAX_CLOSE_NEG, MAX_FAR_NEG)
triplet_test_dataset = TripletAudio(False, K, MAX_CLOSE_NEG, MAX_FAR_NEG)
triplet_train_loader = torch.utils.data.DataLoader(triplet_train_dataset, batch_size=BATCH_SIZE, shuffle=True)
triplet_test_loader = torch.utils.data.DataLoader(triplet_test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [261]:
%autoreload 2
# Set up the network and training parameters
from networks import AnchorNet, EmbeddingNet, TripletNet
from losses import TripletLoss
import torch.optim as optim
from torch.optim import lr_scheduler

INPUT_D, OUTPUT_D = 192, 128
MARGIN, LEARNING_RATE, N_EPOCHS, LOG_INT = 0.5, 1e-3, 5, 1000

#define model
anchor_net = AnchorNet(triplet_train_dataset.get_dataset(), INPUT_D, OUTPUT_D)
embedding_net = EmbeddingNet(anchor_net)
model = TripletNet(embedding_net)

loss_fn = TripletLoss(MARGIN)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)

In [263]:
%autoreload 2
from trainer import fit
fit(triplet_train_loader, triplet_test_loader, model, loss_fn, optimizer, scheduler, N_EPOCHS, {}, LOG_INT)

Train: [0/48048 (0%)]	Loss: 0.488239
Train: [1000/48048 (2%)]	Loss: 2.204387
Train: [2000/48048 (4%)]	Loss: 1.926435
Train: [3000/48048 (6%)]	Loss: 1.405329
Train: [4000/48048 (8%)]	Loss: 1.299205
Train: [5000/48048 (10%)]	Loss: 0.965401
Train: [6000/48048 (12%)]	Loss: 0.723217
Train: [7000/48048 (15%)]	Loss: 0.924041
Train: [8000/48048 (17%)]	Loss: 0.582389
Train: [9000/48048 (19%)]	Loss: 0.627408
Train: [10000/48048 (21%)]	Loss: 0.483703
Train: [11000/48048 (23%)]	Loss: 0.521301
Train: [12000/48048 (25%)]	Loss: 0.366174
Train: [13000/48048 (27%)]	Loss: 0.404740
Train: [14000/48048 (29%)]	Loss: 0.409195
Train: [15000/48048 (31%)]	Loss: 0.450108
Train: [16000/48048 (33%)]	Loss: 0.396983
Train: [17000/48048 (35%)]	Loss: 0.385802
Train: [18000/48048 (37%)]	Loss: 0.328953
Train: [19000/48048 (40%)]	Loss: 0.344354
Train: [20000/48048 (42%)]	Loss: 0.334208
Train: [21000/48048 (44%)]	Loss: 0.307304
Train: [22000/48048 (46%)]	Loss: 0.283614
Train: [23000/48048 (48%)]	Loss: 0.281583
Train: [24

KeyboardInterrupt: 

### Online Selection

In [358]:
# %load_ext autoreload
%autoreload 2
import torch
from datasets import AudioTrainDataset, AudioTestDataset
from datasets import BalancedBatchSampler

K = 5
train_dataset = AudioTrainDataset(K)
test_dataset = AudioTestDataset(K)

train_batch_sampler = BalancedBatchSampler(train_dataset)
test_batch_sampler = BalancedBatchSampler(test_dataset)

online_train_loader = torch.utils.data.DataLoader(train_dataset, batch_sampler=train_batch_sampler)
online_test_loader = torch.utils.data.DataLoader(test_dataset, batch_sampler=test_batch_sampler)

In [355]:
# Set up the network and training parameters
from networks import EmbeddingNet, AnchorNet
from losses import OnlineTripletLoss
from utils import SemihardNegativeTripletSelector # Strategies for selecting triplets within a minibatch
from metrics import AverageNonzeroTripletsMetric

N_EPOCHS, LOG_INT = 20, 25

model = embedding_net
loss_fn = OnlineTripletLoss(MARGIN, SemihardNegativeTripletSelector(MARGIN, train_dataset.KNN))
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)

In [362]:
fit(online_train_loader, online_test_loader, model, loss_fn, optimizer, scheduler, N_EPOCHS, {}, LOG_INT, metrics=[AverageNonzeroTripletsMetric()])

len of triplets 425
Train: [0/48048 (0%)]	Loss: 0.245239	Average nonzero triplets: 425.0
len of triplets 543
len of triplets 525
len of triplets 394
len of triplets 443
len of triplets 430
len of triplets 467
len of triplets 389


KeyboardInterrupt: 